In [1]:
import pandas as pd
import numpy as np
import re
from mysql_helper import _mysql
import warnings
warnings.filterwarnings('ignore')
import pickle

In [2]:
local=_mysql('database-1.cmnqxmfv4x6c.ap-south-1.rds.amazonaws.com','3306','admin','ngasceadmin123','acads')

In [ ]:
# For all columns
# local.fetch_column('session_query_answer')
# local.fetch_query('''SELECT * FROM acads.session_query_answer where createdDate>= "2018-01-01 00:00:00" and createdDate<"2023-01-01 00:00:00"; ''')

In [ ]:
# for only yearly data

In [3]:
local.show_column = ['query','queryType','answer']
local.fetch_query('''SELECT query,queryType,answer FROM acads.session_query_answer where createdDate>= "2018-01-01 00:00:00" and createdDate<"2023-01-01 00:00:00";''')

In [4]:
df = local.show_table

In [5]:
df=df[df['query'].isnull()==False].reset_index(drop=True)
# df=df[df['queryType'].isnull()==False].reset_index(drop=True)
# df=df[df['answer'].isnull()==False].reset_index(drop=True)

In [6]:
# removing querys which hav 2 or less words
to_remove = []
for index,row in df.iterrows():
    if len(row['query'].split(' ')) <=2:
        to_remove.append(index)
df.drop(to_remove,inplace=True)

In [7]:
df['query'] = df['query'].dropna().apply(str.lower)
df['queryType'] = df['queryType'].dropna().apply(str.lower)
df['answer'] = df['answer'].dropna().apply(str.lower)
df['queryType'] = df['queryType'].replace(['course query', 'academic',''], 'non-technical')

In [8]:
df['queryType'].value_counts()

queryType
non-technical    59491
technical         3792
Name: count, dtype: int64

In [9]:
# changing query type of the query

pattern = re.compile(r'''\bKindly restrict\b|\bngasce@nimms.edu\b|\bNGASCE@nmims.edu\b|\bstudent support team\b|\bNGASCE Team\b|\bco- ordinator\b|\bcoordinator\b|\bportal\b|\bexamination department\b|\bpost academic related query\b|\bhelp desk\b|\b1800-1025-136\b<p><b>Kindly restrict  your queries to the session under which you are posting the query. </b></p><p><b>Queries other  than those related to the particular session and/or subject will not be  answered in this section.</b></p><p><b>For all  academic administrative and other queries related to support please approach </b><a><b>NGASCE@nmims.edu</b></a></p>\b|\b<p>Kindly restrict your  queries to the session under which you are posting the query. Queries other  than those related to the particular session and/or subject will not be  answered in this section. For all academic administrative and other queries  related to support please approach NGASCE@nmims.edu</p><p>Regards,</p><p>NGASCE</p>\b|\bKindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu
Regards,
NGASCE\b|\bKindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu\b|\b<b>please approach </b><a><b>NGASCE@nmims.edu</b></a>\b|\bplease  approach NGASCE@nmims.edu\b|\b<p><b>&ldquo;Kindly restrict your queries to the session under  which you are &nbsp;&nbsp;&nbsp;posting the query.</b></p><p><b>&nbsp;</b></p><p><b>Queries other than those related to the particular  session and/or subject will not be answered in this section.</b></p><p><b> </b></p><p><b>For all academic administrative and other queries  related to support please do follow the below given steps;</b></p><p><b> </b></p><p><b>Click on the tab, Student Support &gt; </b><b>Connect  With Us &gt; Raise a Ticket {My Tickets} &gt; Raise a Ticket &gt; fill in your  details &gt; Submit&rdquo;</b></p>\b|\bFor all academic administrative and other queries related to support please approach NGASCE@nmims.edu\b|\b<p><b>&ldquo;Kindly  restrict your queries to the session under which you are &nbsp;&nbsp;&nbsp;posting the query. </b></p><p><b>Queries other  than those related to the particular session and/or subject will not be  answered in this section.</b></p><p><b>For all  academic administrative and other queries related to support please approach </b><a><b>NGASCE@nmims.edu</b></a><b> </b><b>&rdquo;</b></p>\b|\bplease write to ngasce@nmims.edu\b|\b<p>Kindly restrict your queries to the session under which you are  posting the query. Queries other than those related to the particular session  and/or subject will not be answered in this section. For all academic  administrative and other queries related to support please approach NGASCE@nmims.edu</p><p>Regards,</p><p>NGASCE</p>\b|\bKindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu

Regards,
NGASCE\b|\bFor  all academic administrative and other queries related to support please  approach NGASCE@nmims.edu\b|\bKindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach ngasce@nmims.edu

Regards,\b|\b<p><b>&ldquo;Kindly  restrict your queries to the session under which you are &nbsp;&nbsp;&nbsp;posting the query.</b></p><p><b>&nbsp;</b></p><p><b>Queries other  than those related to the particular session and/or subject will not be  answered in this section.</b></p><p><b> </b></p><p><b>For all  academic administrative and other queries related to support please do follow  the below given steps;</b></p><p><b> </b></p><p><b>Click on the tab, Student Support &gt; </b><b>Connect  With Us &gt; Raise a Ticket {My Tickets} &gt; Raise a Ticket &gt; fill in your  details &gt; Submit</b><b>&rdquo;</b></p>\b|\b<p><b>For all  academic administrative and other queries related to support please approach </b><a><b>NGASCE@nmims.edu</b></a></p>\b|\b<p>Kindly restrict your queries to  the session under which you are posting the query. Queries other than those  related to the particular session and/or subject will not be answered in this  section. For all academic administrative and other queries related to support  please approach <a><b>NGASCE@nmims.edu</b></a></p><p>Regards,</p><p>NGASCE</p>\b|\b<a href="https://adhocfilesngasce.s3.ap-south-1.amazonaws.com/academics/1655290709OMeYb.pdf">https://adhocfilesngasce.s3.ap-south-1.amazonaws.com/academics/1655290709OMeYb.pdf</a>\b|\b<!--StartFragment--><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">The Individual Task is uploaded in the&nbsp;<b>Learning Resources Section</b>. All the details are mentioned in the document itself.</p><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">You have to&nbsp;<b>Upload your work as a PDF file in the Upload Assignment Section</b>. If you don&rsquo;t see this on your portal</p><!--EndFragment-->\b|\b<p>Kindly restrict your  queries to the session under which you are posting the query. Queries other  than those related to the particular session and/or subject will not be  answered in this section. For all academic administrative and other queries  related to support please approach NGASCE@nmims.edu</p>\b|\b. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu
Regards,
NGASCE\b|\bKindly mail your admission, exam-related queries on ngasce@nimms.edu\b|\bPlease approach NGASCE@nmims.edu\b|\b<!--StartFragment-->Plz refer&nbsp;chapter 5 section 5.3.1 and 5.3.2<br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">3 LAWS:&nbsp;<br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">Moore&apos;s law,&nbsp; law of mass digital storage , Metcalfe&apos;s law.<br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">Plz elaborate on these.<br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">Regards<br style="box-sizing: border-box; margin: 0px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;">Jhalak M<!--EndFragment-->\b|\b<p>The Individual Task is already uploaded on the  student&rsquo;s portal in the Upload Assignments Section as a PDF File.</p><p>If you don&rsquo;t see this on your portal, please connect  to the Admin Desk at NMIMS.</p>\b|\bPlease email your query to NGASCE@nmims.edu\b|\b<!--StartFragment-->Kindly raise a ticket so that you can get the title approval for the subject expert. Also, please go through the project session recordings.<!--EndFragment-->\b|\bDear Student,
Please use this forum for session related queries for other concerns please get in touch with ngasce@nmims.edu\b|\bKindly forward your query to ngasce@nmims.edu\b|\bPlease write to NGASCE@nmims.edu.\b|\b<p>Kindly restrict your queries to the session under which you are &nbsp;&nbsp;&nbsp;posting the query.</p><p>&nbsp;</p><p>Queries other than those related to the particular session and/or subject  will not be answered in this section.</p><p> </p><p>For all academic administrative and other queries related to support please  do follow the below given steps;</p><p> </p><p>Click on the tab, Student Support &gt; Connect With Us &gt; Raise a Ticket {My Tickets} &gt;  Raise a Ticket &gt; fill in your details &gt; Submit.</p><p> </p><p> </p><p>Regards,</p><p>NGASCE  Academics</p>\b|\bPLease contact course coordinator\b|\b<!--StartFragment--><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><b>&ldquo;Kindly restrict your queries to the session under which you are &nbsp;&nbsp;&nbsp;posting the query.</b></p><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><b>&nbsp;</b></p><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><b>Queries other than those related to the particular session and/or subject will not be answered in this section.</b></p><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><b>For all academic administrative and other queries related to support please do follow the below given steps;</b></p><p style="box-sizing: border-box; margin: 0px 0px 10px; padding: 0px; color: rgb(51, 51, 51); font-family: &amp;quot;Trebuchet MS&amp;quot;, Tahoma, Verdana, Arial, sans-serif; font-size: 15.4px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial;"><b>Click on the tab, Student Support &gt;&nbsp;</b><b>Connect With Us &gt; Raise a Ticket {My Tickets} &gt; Raise a Ticket &gt; fill in your details &gt; Submit&rdquo;</b></p><!--EndFragment-->\b|\b<p><b>&ldquo;Kindly  restrict your queries to the session under which you are &nbsp;&nbsp;&nbsp;posting the query.</b></p><p><b>&nbsp;</b></p><p><b>Queries other  than those related to the particular session and/or subject will not be  answered in this section.</b></p><p><b> </b></p><p><b>For all  academic administrative and other queries related to support please do follow  the below given steps;</b></p><p><b> </b></p><p><b>Click on the tab, Student Support &gt; </b><b>Connect  With Us &gt; Raise a Ticket {My Tickets} &gt; Raise a Ticket &gt; fill in your  details &gt; Submit</b><b>&nbsp; </b></p>\b|\bThis portal is for subject related query\b|\b<p><b>&ldquo;Kindly restrict your queries to the session under  which you are &nbsp;&nbsp;&nbsp;posting the query.</b></p><p><b>&nbsp;</b></p><p><b>Queries other than those related to the particular  session and/or subject will not be answered in this section.</b></p><p><b> </b></p><p><b>For all academic administrative and other queries  related to support please do follow the below given steps;</b></p><p><b> </b></p><b>Click on the tab, Student Support &gt; </b><b>Connect With Us &gt; Raise a Ticket {My Tickets} &gt; Raise a Ticket  &gt; fill in your details &gt; Submit&rdquo;</b>\b|\b<p><b>please approach  </b><a><b>NGASCE@nmims.edu</b></a></p>\b|\bLog in with Student Id->>  My Courses->>   Click on the respective Subject ->>  Click on Resources.\b|\b<p>Kindly restrict your queries to the session under which you are posting  the query. Queries other than those related to the particular session and/or  subject will not be answered in this section. For all academic administrative  and other queries related to support please approach NGASCE@nmims.edu</p><p>Regards,</p><p>NGASCE</p>\b|\bplease contact your course coordinator\b|\b<p>Kindly restrict your queries to the session under which you are  posting the query. Queries other than those related to the particular session  and/or subject will not be answered in this section. For all academic  administrative and other queries related to support please approach NGASCE@nmims.edu</p><p>Regards,</p><p>NGASCE</p><p> </p>\b|\bplease connect student support team\b|\bPlease connect with NGASCE Team\b|\b&ldquo;Kindly restrict your queries to the session under which you are<br>posting the query.<br>Queries other than those related to the particular session and/or<br>subject will not be answered in this section.<br>For all academic administrative and other queries related to<br>support please do follow the below given steps;<br>Click on the tab, Student Support &gt; Connect With Us &gt; Raise<br>a Ticket {My Tickets} &gt; Raise a Ticket &gt; fill in your details<br>&gt; Submit&rdquo;\b|\bKindly restrict your queries to the session under which you are posting the query. 
Queries other than those related to the particular session and/or subject will not be answered in this section.
For all academic administrative and other queries related to support please approach NGASCE@nmims.edu\b|\bReply :

Kindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu\b|\b<p><b>&ldquo;Kindly restrict your queries to the session under  which you are &nbsp;&nbsp;&nbsp;posting the query.</b></p><p><b>&nbsp;</b></p><p><b>Queries other than those related to the particular  session and/or subject will not be answered in this section.</b></p><p><b> </b></p><p><b>For all academic administrative and other queries  related to support please do follow the below given steps;</b></p><p><b> </b></p><p><b>Click on the tab, Student Support &gt; </b><b>Connect  With Us &gt; Raise a Ticket {My Tickets} &gt; Raise a Ticket &gt; fill in your  details &gt; Submit&rdquo;</b></p><p><b> </b></p>\b|\b<p>Kindly restrict your queries to  the session under which you are posting the query. Queries other than those  related to the particular session and/or subject will not be answered in this  section. For all academic administrative and other queries related to support  please approach <a><b>NGASCE@nmims.edu</b></a></p><p>Regards,</p><p>NGASCE</p><p> </p>\b|\bIT will be uploaded soon.\b|\bPlease connect with your academic coordinator\b|\bThis portal is only for subject related query\b|\b<p> </p><p><b>Kindly restrict  your queries to the session under which you are posting the query. </b></p><p><b>Queries other  than those related to the particular session and/or subject will not be  answered in this section.</b></p><p><b>For all  academic administrative and other queries related to support please approach </b><a><b>NGASCE@nmims.edu</b></a></p>\b|\b<p><b>Kindly restrict your queries to the session under which you are posting the query.</b></p><p><b>Queries other than those related to the particular session and/or subject will not be answered in this section.</b></p><p><b>For all academic administrative and other queries related to support please approach&nbsp;NGASCE@nmims.edu</b></p>\b|\bKindly restrict your queries to the session under which you are    posting the query.
 
Queries other than those related to the particular session and/or subject will not be answered in this section.

For all academic administrative and other queries related to support please do follow the below given steps;

Click on the tab, Student Support > Connect With Us > Raise a Ticket {My Tickets} > Raise a Ticket > fill in your details > Submit\b|\bKindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu

Regards\b|\bKindly write to ngasce@nmims.edu\b|\bPlease contact the course coordinator.\b|\bPlease get in touch with NGASCE@nmims.edu\b|\bDear Student,
Kindly connect with your academic coordinator for such queries. 
Regards\b|\bDear Student,
Kindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu

Regards,\b|\bkindly connect to support team\b|\bKindly post this query to exam team\b|\bKindly restrict your queries to the session under which you are posting the query. Queries other than those related to the particular session and/or subject will not be answered in this section. For all academic administrative and other queries related to support please approach NGASCE@nmims.edu

Regards,

NGASCE\b|\bPlease post your administrative related queries as per the details below.

Write to us at NGASCE@nmims.edu
You can also call us between 9 AM and 7 PM, Monday to Saturday, on 1800-1025-136.\b|\bpls get in touch with NGASCE.\b|\bThis portal is to be used only for asking subject related queries\b|\b&nbsp;please approach NGASCE@nmims.edu\b|\b<p>Kindly restrict your queries to  the session under which you are posting the query. Queries other than those  related to the particular session and/or subject will not be answered in this  section. For all academic administrative and other queries related to support  please approach <a><b>NGASCE@nmims.edu</b></a></p><p>Regards,</p>NGASCE\b''')

for index,row in df.iterrows():
    if len(pattern.findall(str(row['answer'])))!=0:
        df.loc[index,"queryType"]='technical'

In [11]:
df['queryType'].value_counts()

queryType
non-technical    57534
technical         5772
Name: count, dtype: int64

In [ ]:
#words_for_tech_query = ['Kindly restrict','ngasce@nimms.edu','NGASCE@nmims.edu','student support team','NGASCE Team','co- ordinator','coordinator','portal','examination department','post academic related query','help desk','1800-1025-136','technical','escalated','admin']

# a=''
# for i in words_for_tech_query:
#     print('\\b'+i+'\\b|',end='')


# to_clean = pd.read_csv(r'C:\Users\Pramod.gunaware.EXT\Downloads\clean.csv')
# for i in to_clean['answer']:
# #     print("'''"+i+"''',")
#     print('\\b'+i+'\\b|',end='')

In [12]:
# no need of an answeer from here
df=df[['query','queryType']]

In [13]:
query_df = df

In [14]:
query_df['query']=query_df['query'].str.lower()
query_df['queryType']=query_df['queryType'].str.lower()

In [15]:
dict_={"abt" : "about",  
"acc" : "acount" ,
"aka" : "also known as",
"asap" : "as soon as possible",
"b4" : "before",
"btw" : "by the way",
"ctn" : "cant talk now",
"cu" : "see you",
"cua" : "see you around",
"cul" : "see you later",
"cya" : "see ya",
"da" : "the",
"dnt" : "Dont",
"ema" : "Email address",
"eta"  : "Estimated time of arrival",
"ez" : "Easy",
"faq" : "Frequently Asked Questions",
"fb" : "Facebook",
"fyi" : "For your information",
"gud" : "Good",
"hv" : "Have",
"hw" : "Homework",
"idk" : "I dont know",
"im" : "i am",
"msg" : "Message",
"n/a" : "Not available or not applicable",
"nvr" : "Never",
"omg" : "Oh my God",
"omw" : "On my way",
"pls" : "Please",
"plz" : "Please",
"pov" : "Point of view",
"ppl" : "People",
"prolly" : "Probably",
"pwned" : "Owned, as in conquered or defeated",
"sec" : "Second",
"sms" : "Short Message Service",
"sry" : "Sorry",
"tbh" : "To be honest",
"thanq" : "Thank you",
"thx" : "Thanks",
"tweet" : "Twitter post",
"txt" : "Text",
"ty" : "Thank you",
"u" : "You",
"ur" : "Your",
"w8" : "Wait",
"wrk" : "Work",
"yr" : "year",
"24/7" : "seven days a week"}



def slang_convert(text):
    new_text=[]
    for word in text.split():
        if word.lower() in dict_:
            new_text.append(dict_[word.lower()].lower())
        else:
            new_text.append(word)
    return " ".join(new_text)

query_df["query"]=query_df["query"].apply(slang_convert)

In [16]:
#remove punctuation
import string
def remove_punc(txt):
    for i in string.punctuation:
        txt=txt.replace(i,'')
    return txt

query_df['query_clean']=query_df['query'].apply(remove_punc)

In [17]:
def preprocessing(query):
    query = str(query)
    # removes query with 2 or less words
    if len(query.split(' ')) <=3:
        del(query)
        pass
    else:
        query=query.lower()
        query = slang_convert(query)
        query = remove_punc(query)
        query = remove_emoji(query)

In [18]:
# remove emojis
import emoji
def remove_emoji(text):
    return(emoji.demojize(text).replace(':',''))

query_df['query_clean']=query_df['query_clean'].apply(remove_emoji)

In [19]:
import gensim
from nltk import sent_tokenize
#from gensim.utils import simple_preprocess
from nltk import word_tokenize

In [20]:
corpus=[]
for doc in query_df['query_clean']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        corpus.append(word_tokenize(sent))

In [21]:
model = gensim.models.Word2Vec(window=10,min_count=2)

In [22]:
model.build_vocab(corpus)

In [23]:
model.train(corpus,total_examples=model.corpus_count, epochs=model.epochs)

(7669110, 10523970)

In [ ]:

pickle.dump(model,open('ngasce_word2vecV2.pkl','wb'))

In [ ]:
model = pickle.load(open('ngasce_word2vecV2.pkl','rb'))

In [24]:
model.wv.most_similar('session')

[('lecture', 0.8872261047363281),
 ('video', 0.7874114513397217),
 ('class', 0.765842080116272),
 ('sessions', 0.7651443481445312),
 ('recording', 0.7064437866210938),
 ('seesion', 0.6789711713790894),
 ('track', 0.6667041182518005),
 ('mins', 0.6636064052581787),
 ('minutes', 0.6538895964622498),
 ('recordings', 0.653363823890686)]

In [25]:
def doc_vec(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [26]:
query_df

,query,queryType,query_clean
0,when are the classes for cpm starting in janua...,non-technical,when are the classes for cpm starting in janua...
1,please let me know when result for my first se...,non-technical,please let me know when result for my first se...
2,"dear sir/madam, i need the reference material ...",non-technical,dear sirmadam i need the reference material fo...
3,"hello sir, would it be possible for you to mee...",non-technical,hello sir would it be possible for you to meet...
5,"all right sir, what would be your contact no?",non-technical,all right sir what would be your contact no
...,...,...,...
63697,having doubt in 5 purpose of business law,non-technical,having doubt in 5 purpose of business law
63698,if both butterflies and strangers short term c...,non-technical,if both butterflies and strangers short term c...
63699,maam efficient market hypothesis is just a the...,non-technical,maam efficient market hypothesis is just a the...
63700,what is meaning of anchor investor in accounti...,non-technical,what is meaning of anchor investor in accounti...


In [ ]:
y  # target

In [ ]:
x   # input

In [27]:
from tqdm import tqdm

x=[]
for doc in tqdm(query_df['query_clean'].values):
    x.append(doc_vec(doc))

100%|██████████| 63509/63509 [01:06<00:00, 948.62it/s] 


In [28]:
x = np.array(x)

In [ ]:
x

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [30]:
y=encoder.fit_transform(query_df['queryType'])

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [32]:
x_train = np.array(x_train)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [34]:
rf =RandomForestClassifier()

In [35]:
rf.fit(np.array(x),y)

RandomForestClassifier()

In [36]:
y_pred=(rf.predict(x_test))

In [37]:
accuracy_score(y_test,y_pred)

0.9960636120296016

In [ ]:
query_df

In [ ]:
y

In [ ]:
a=np.array((doc_vec('''17th session not clear at the end''')))

rf.predict(a.reshape(1, -1))

In [ ]:
query_df[query_df['query']=='I wanted to ask query regarding Exam registration- what is the meaning of first come first serve basis registration. Is there a possibility someone will not get to sit in exam even after completing 6 months of learning?'.lower()]

In [ ]:
print(gensim.__version__)

In [38]:
pickle.dump(model,open('ngasce_word2vec.pkl','wb'))
pickle.dump(rf,open('rf_classifier.pkl','wb'))

In [ ]:
df[df['parentid']!='xyz']